# 動態網頁爬蟲 - 使用 Selenium

* 了解 Selenium 用於動態網頁爬蟲的原理
* 能夠使用 Selenium 撰寫動態網頁爬蟲

## 作業目標


* 根據範例使用 selenuim 取出 高雄市小港區 2019 年 空氣品質 資料，並用一個適合的資料結構儲存
* Hint: 將所有資料存在一個變數當中



In [1]:
from selenium import webdriver
from selenium.webdriver.support.ui import Select
from bs4 import BeautifulSoup

browser = webdriver.Chrome(executable_path='chromedriver')
browser.get("http://taqm.epa.gov.tw/taqm/tw/MonthlyAverage.aspx")

selectSite = Select(browser.find_element_by_id("ctl05_ddlSite"))
selectSite.select_by_value('58')
selectYear = Select(browser.find_element_by_id("ctl05_ddlYear"))
selectYear.select_by_value('2019')


browser.find_element_by_id('ctl05_btnQuery').click()

![](https://imgur.com/LwGGT8c.jpg)

In [3]:
# 取得資料（等瀏覽器資料出現後才執行）
html_source = browser.page_source
print(html_source[:3000])

<html xmlns="http://www.w3.org/1999/xhtml" lang="zh-TW"><head id="Head1"><base href="https://taqm.epa.gov.tw/taqm/tw/"><meta http-equiv="X-UA-Compatible" content="IE=edge"><meta http-equiv="Content-Type" content="text/html; charset=UTF-8"><title>行政院環境保護署－空氣品質監測網 - 月平均值查詢</title><meta name="title" content="空氣品質監測網(TAQM)">
<!--link rel="govtwsimpledc.xsd" href="http://www.gov.tw/schema/dc/" title="Dublin Core" /--> 
<link type="text/css" href="/taqm/tw/style/all.min.css?v=20180508c" rel="stylesheet">
<script type="text/javascript" async="" src="https://www.google-analytics.com/analytics.js"></script><script type="text/javascript" src="/taqm/script/jquery.min.js"></script>
<script type="text/javascript" src="/taqm/script/jquery.myslide2.min.js"></script>
<script type="text/javascript">
function DownloadReport(url) {
	var win = window.open("DownloadReport.aspx?file="+escape(url),"download" /*,"modal=yes,menubar=no,scrollbars=no,status=no,titlebar=no,toolbar=no,height=200px,width=300px"*/);

In [4]:
soup = BeautifulSoup(html_source, 'html.parser')
table = soup.find('table', class_='TABLE_G')
table

<table align="Center" border="1" cellpadding="3" cellspacing="0" class="TABLE_G" id="ctl05_gv" rules="all" style="border-color:Black;border-width:1px;border-style:None;border-collapse:collapse;">
<tbody><tr style="color:Black;font-weight:normal;">
<th scope="col">監測項目</th><th scope="col">單位</th><th scope="col">監測日期</th><th scope="col">監測值</th><th scope="col">標註</th>
</tr><tr style="color:Black;">
<td class="no-alt" rowspan="11" style="white-space:nowrap;" valign="top">SO2</td><td class="no-alt" rowspan="11" valign="top">ppb</td><td>2019/01</td><td>4.10</td><td> </td>
</tr><tr class="ALT" style="color:Black;">
<td>2019/02</td><td>3.30</td><td> </td>
</tr><tr style="color:Black;">
<td>2019/03</td><td>3.90</td><td> </td>
</tr><tr class="ALT" style="color:Black;">
<td>2019/04</td><td>3.40</td><td> </td>
</tr><tr style="color:Black;">
<td>2019/05</td><td>3.30</td><td> </td>
</tr><tr class="ALT" style="color:Black;">
<td>2019/06</td><td>4.10</td><td> </td>
</tr><tr style="color:Black;">
<td>

In [59]:
#找出所有監測目標＋整理到elementList中dimlists=[]
dcnt=0
dimlists={}
for d in table.find_all(['td'],attrs={"style":"white-space:nowrap;"}):
    #display(d.string)
    #display(d.get("rowspan"))
    #dimlists.append(d.string)
    dimlists[d.string]=d.get("rowspan")
    #if not(dcnt%35) :
        #print('tt')
print(dimlists)


{'SO2': '11', 'CO': '11', 'O3': '11', 'PM10': '11', 'NOx': '11', 'NO': '11', 'NO2': '11', 'THC': '11', 'NMHC': '11', 'CH4': '11'}


In [88]:
measurings=dict()
dimcnt = 0
meacnt = 0
for dim,measure in dimlists.items():
    d1={}
    dailymeasures={}
    #print('dim=',dim,' measure=',measure)
    #getmeasure=table.find(['td'],dimlist.key)
    #print(getmeasure)
    
    for dailymeasure in range(11):
        date = table.find_all('td')[35*dimcnt + 2 + 3*meacnt].contents[0] #監測日期
        content = table.find_all('td')[35*dimcnt + 3 + 3*meacnt].contents[0] #監測量
        #print('date=',date,' content=',content)
        d1[date] = content
        meacnt = meacnt + 1
    measurings[dim] = d1
    meacnt = 0
    dimcnt = dimcnt + 1

print(measurings)


{'SO2': {'2019/01': '4.10', '2019/02': '3.30', '2019/03': '3.90', '2019/04': '3.40', '2019/05': '3.30', '2019/06': '4.10', '2019/07': '4', '2019/08': '4.60', '2019/09': '3.50', '2019/10': '3.20', '2019/11': '3.60'}, 'CO': {'2019/01': '0.67', '2019/02': '0.48', '2019/03': '0.50', '2019/04': '0.39', '2019/05': '0.38', '2019/06': '0.24', '2019/07': '0.28', '2019/08': '0.33', '2019/09': '0.32', '2019/10': '0.36', '2019/11': '0.46'}, 'O3': {'2019/01': '24.70', '2019/02': '29.40', '2019/03': '29.20', '2019/04': '30.40', '2019/05': '28.60', '2019/06': '16.30', '2019/07': '15.50', '2019/08': '19.70', '2019/09': '35.70', '2019/10': '44', '2019/11': '34.80'}, 'PM10': {'2019/01': '78', '2019/02': '57', '2019/03': '52', '2019/04': '42', '2019/05': '35', '2019/06': '20', '2019/07': '24', '2019/08': '27', '2019/09': '35', '2019/10': '50', '2019/11': '63'}, 'NOx': {'2019/01': '38.01', '2019/02': '25.62', '2019/03': '27.84', '2019/04': '21.83', '2019/05': '22.55', '2019/06': '17.15', '2019/07': '17.71

In [90]:
import pandas as pd
pd.DataFrame(measurings)

,SO2,CO,O3,PM10,NOx,NO,NO2,THC,NMHC,CH4
2019/01,4.10,0.67,24.70,78,38.01,7.95,30.06,2.27,0.27,2
2019/02,3.30,0.48,29.40,57,25.62,4.76,20.86,2.11,0.17,1.94
2019/03,3.90,0.50,29.20,52,27.84,5.41,22.43,2.12,0.18,1.94
2019/04,3.40,0.39,30.40,42,21.83,4.16,17.68,2.02,0.14,1.88
2019/05,3.30,0.38,28.60,35,22.55,4.58,17.98,2.02,0.16,1.86
2019/06,4.10,0.24,16.30,20,17.15,5.08,12.08,1.92,0.13,1.79
2019/07,4,0.28,15.50,24,17.71,5.16,12.54,1.93,0.14,1.80
2019/08,4.60,0.33,19.70,27,19.66,5.17,14.49,1.97,0.15,1.82
2019/09,3.50,0.32,35.70,35,18.66,3.63,15.03,1.98,0.13,1.86
2019/10,3.20,0.36,44,50,19.50,3.50,15.99,2.05,0.13,1.92
